In [1]:
from huggingface_hub import HfApi, create_repo, upload_file
import os

HF_TOKEN = os.environ.get("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("HF_TOKEN not found. Set it as an environment variable.")

username = "Disha252001"
# Use a simple, valid repo name on Hugging Face (no spaces or parentheses)
dataset_name = "tourism-1"
repo_id = f"{username}/{dataset_name}"

# Path to the actual CSV in your workspace
local_file = "master_folder/data/tourism (1).csv"

api = HfApi()

# Create repo if it doesn't exist
create_repo(
    repo_id=repo_id,
    repo_type="dataset",
    token=HF_TOKEN,
    exist_ok=True
)

# Upload raw CSV into /raw folder in HF dataset
upload_file(
    path_or_fileobj=local_file,
    path_in_repo="raw/tourism (1).csv",
    repo_id=repo_id,
    repo_type="dataset",
    token=HF_TOKEN,
    commit_message="Upload raw tourism dataset"
)

print("✔ Raw dataset uploaded successfully.")



c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No files have been modified since last commit. Skipping to prevent empty commit.


✔ Raw dataset uploaded successfully.


In [2]:
from datasets import load_dataset

repo_id = "Disha252001/tourism-1"

dataset = load_dataset(repo_id)
df = dataset["train"].to_pandas()

df.head()


Generating test split: 100%|██████████| 826/826 [00:00<00:00, 71104.49 examples/s]


,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,201923,1,26.0,Company Invited,2,23.0,Salaried,Female,2,3.0,Basic,3.0,Married,1.0,1,5,0,1.0,Executive,17741.0
1,200597,0,42.0,Self Enquiry,1,6.0,Salaried,Female,2,4.0,Deluxe,3.0,Divorced,1.0,1,3,0,1.0,Manager,21062.0
2,204701,0,56.0,Company Invited,1,9.0,Salaried,Male,4,4.0,Standard,4.0,Married,5.0,0,1,0,2.0,Senior Manager,29654.0
3,204662,1,27.0,Company Invited,3,36.0,Small Business,Male,4,6.0,Deluxe,5.0,Unmarried,2.0,0,3,1,2.0,Manager,23647.0
4,202821,0,37.0,Self Enquiry,1,9.0,Salaried,Female,4,4.0,Basic,3.0,Divorced,6.0,0,5,1,1.0,Executive,21221.0


In [3]:
# Remove unnamed index column if present
df_clean = df.loc[:, ~df.columns.str.contains("Unnamed")]

# Drop any completely empty columns
df_clean = df_clean.dropna(axis=1, how="all")

# Example: drop CustomerID if unnecessary
# df_clean = df_clean.drop(columns=["CustomerID"])

df_clean.head()


,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,201923,1,26.0,Company Invited,2,23.0,Salaried,Female,2,3.0,Basic,3.0,Married,1.0,1,5,0,1.0,Executive,17741.0
1,200597,0,42.0,Self Enquiry,1,6.0,Salaried,Female,2,4.0,Deluxe,3.0,Divorced,1.0,1,3,0,1.0,Manager,21062.0
2,204701,0,56.0,Company Invited,1,9.0,Salaried,Male,4,4.0,Standard,4.0,Married,5.0,0,1,0,2.0,Senior Manager,29654.0
3,204662,1,27.0,Company Invited,3,36.0,Small Business,Male,4,6.0,Deluxe,5.0,Unmarried,2.0,0,3,1,2.0,Manager,23647.0
4,202821,0,37.0,Self Enquiry,1,9.0,Salaried,Female,4,4.0,Basic,3.0,Divorced,6.0,0,5,1,1.0,Executive,21221.0


In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_clean, test_size=0.2, random_state=42)

train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("Train and test CSVs saved locally!")


Train and test CSVs saved locally!


In [5]:
from huggingface_hub import upload_file
import os

HF_TOKEN = os.environ["HF_TOKEN"]
repo_id = "Disha252001/tourism-1"

# Upload train.csv
upload_file(
    path_or_fileobj="train.csv",
    path_in_repo="processed/train.csv",
    repo_id=repo_id,
    repo_type="dataset",
    token=HF_TOKEN
)

# Upload test.csv
upload_file(
    path_or_fileobj="test.csv",
    path_in_repo="processed/test.csv",
    repo_id=repo_id,
    repo_type="dataset",
    token=HF_TOKEN
)

print("Processed train/test uploaded to HuggingFace!")


Processed train/test uploaded to HuggingFace!


In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import joblib
from huggingface_hub import upload_file
import os
from datasets import load_dataset

# Hugging Face credentials
HF_TOKEN = os.environ["HF_TOKEN"]
repo_id = "Disha252001/tourism-1"

# Load data
dataset = load_dataset(repo_id, data_files={"train": "processed/train.csv", "test": "processed/test.csv"})
train_df = dataset["train"].to_pandas()
test_df = dataset["test"].to_pandas()

# Target column
TARGET = "ProdTaken"
DROP_COLS = ["CustomerID"]   # <-- IMPORTANT

# Split features & target
X_train = train_df.drop([TARGET] + DROP_COLS, axis=1)
y_train = train_df[TARGET]

X_test = test_df.drop([TARGET] + DROP_COLS, axis=1)
y_test = test_df[TARGET]

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ],
    remainder='passthrough'  # numeric columns unchanged
)



In [30]:
# Models and hyperparameters
models_params = {
    "DecisionTree": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {"classifier__max_depth": [None, 5, 10, 20],
                   "classifier__min_samples_split": [2, 5, 10]}
    },
    "Bagging": {
        "model": BaggingClassifier(random_state=42),
        "params": {"classifier__n_estimators": [10, 50, 100],
                   "classifier__max_samples": [0.5, 0.7, 1.0]}
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {"classifier__n_estimators": [100, 200],
                   "classifier__max_depth": [None, 10, 20],
                   "classifier__min_samples_split": [2, 5]}
    },
    "AdaBoost": {
        "model": AdaBoostClassifier(random_state=42),
        "params": {"classifier__n_estimators": [50, 100, 200],
                   "classifier__learning_rate": [0.01, 0.1, 1.0]}
    },
    "GradientBoosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {"classifier__n_estimators": [100, 200],
                   "classifier__learning_rate": [0.01, 0.1, 0.2],
                   "classifier__max_depth": [3, 5, 10]}
    },
    "XGBoost": {
        "model": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="mlogloss"),
        "params": {"classifier__n_estimators": [100, 200],
                   "classifier__learning_rate": [0.01, 0.1],
                   "classifier__max_depth": [3, 5, 10]}
    }
}


In [31]:
from huggingface_hub import create_repo

# Create a new model repo
model_repo_id = "Disha252001/tourism-1-models"
create_repo(repo_id=model_repo_id, repo_type="model", token=HF_TOKEN)
print(f"Model repo created: {model_repo_id}")


HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-693aad43-63a26f8f38509e515fa3f30b;7dfad18e-7cbb-40d1-b588-e678b7b120c0)

You already created this model repo: Disha252001/tourism-1-models

In [32]:
# Train models and upload to Hugging Face model repo
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import joblib
from huggingface_hub import upload_file

# Update with your new model repo
model_repo_id = "Disha252001/tourism-1-models"  # your newly created model repo

best_models = {}

for name, mp in models_params.items():
    print(f"\n=== Training {name} ===")
    
    # Create pipeline with preprocessing
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', mp["model"])])
    
    # Grid search for hyperparameter tuning
    grid = GridSearchCV(pipeline, mp["params"], cv=5, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"{name} Best Params:", grid.best_params_)
    print(f"{name} Test Accuracy:", acc)
    print(classification_report(y_test, y_pred))
    
    # Save locally
    model_filename = f"{name}_best_model.joblib"
    joblib.dump(best_model, model_filename)
    
    # Upload to Hugging Face model hub
    upload_file(
        path_or_fileobj=model_filename,
        path_in_repo=f"models/{model_filename}",  # optional folder inside the repo
        repo_id=model_repo_id,
        repo_type="model",
        token=HF_TOKEN
    )
    
    # Store best model info
    best_models[name] = {"model": best_model, "accuracy": acc, "params": grid.best_params_}

# Identify the overall best model
best_model_name = max(best_models, key=lambda x: best_models[x]["accuracy"])
print(f"\n🏆 Best Model: {best_model_name} with accuracy {best_models[best_model_name]['accuracy']}")



=== Training DecisionTree ===
Fitting 5 folds for each of 12 candidates, totalling 60 fits
DecisionTree Best Params: {'classifier__max_depth': None, 'classifier__min_samples_split': 2}
DecisionTree Test Accuracy: 0.8729198184568835
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       524
           1       0.71      0.66      0.68       137

    accuracy                           0.87       661
   macro avg       0.81      0.80      0.80       661
weighted avg       0.87      0.87      0.87       661



DecisionTree_best_model.joblib: 100%|██████████| 52.8k/52.8k [00:01<00:00, 34.0kB/s]



=== Training Bagging ===
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Bagging Best Params: {'classifier__max_samples': 1.0, 'classifier__n_estimators': 100}
Bagging Test Accuracy: 0.8986384266263238
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       524
           1       0.86      0.61      0.71       137

    accuracy                           0.90       661
   macro avg       0.88      0.79      0.83       661
weighted avg       0.90      0.90      0.89       661



Bagging_best_model.joblib: 100%|██████████| 3.70M/3.70M [00:05<00:00, 648kB/s] 



=== Training RandomForest ===
Fitting 5 folds for each of 12 candidates, totalling 60 fits
RandomForest Best Params: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
RandomForest Test Accuracy: 0.8910741301059002
              precision    recall  f1-score   support

           0       0.89      0.98      0.93       524
           1       0.90      0.53      0.67       137

    accuracy                           0.89       661
   macro avg       0.90      0.76      0.80       661
weighted avg       0.89      0.89      0.88       661



...


=== Training XGBoost ===
Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:10:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Best Params: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 10, 'classifier__n_estimators': 200}
XGBoost Test Accuracy: 0.8986384266263238
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       524
           1       0.84      0.63      0.72       137

    accuracy                           0.90       661
   macro avg       0.88      0.80      0.83       661
weighted avg       0.90      0.90      0.89       661



XGBoost_best_model.joblib: 100%|██████████| 956k/956k [00:02<00:00, 368kB/s]



🏆 Best Model: GradientBoosting with accuracy 0.913767019667171


In [33]:
import os
os.makedirs("master_folder/deployment", exist_ok=True)


In [15]:
%%writefile master_folder/deployment/Dockerfile
# -------------------------
#  Base Python Image
# -------------------------
FROM python:3.10-slim

# -------------------------
#  Setup Working Directory
# -------------------------
WORKDIR /app

# -------------------------
#  Copy Project Files
# -------------------------
COPY . /app

# -------------------------
#  Install Dependencies
# -------------------------
RUN pip install --no-cache-dir --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# -------------------------
#  Expose Streamlit Port
# -------------------------
EXPOSE 8501

# -------------------------
#  Run Streamlit App
# -------------------------
CMD ["streamlit", "run", "app.py", \
     "--server.port=8501", \
     "--server.address=0.0.0.0"]


Writing master_folder/deployment/Dockerfile


In [16]:
%%writefile master_folder/deployment/requirements.txt
streamlit
huggingface_hub
joblib
pandas
scikit-learn
numpy
xgboost


Writing master_folder/deployment/requirements.txt


In [34]:
%%writefile master_folder/deployment/app.py
import os
import streamlit as st
import pandas as pd
import joblib
from huggingface_hub import hf_hub_download
from pathlib import Path

st.set_page_config(page_title="Wellness Tourism Purchase Predictor", layout="centered")

# --------------------------
# Configuration
# --------------------------
# Model repo that you uploaded earlier (change if different)
MODEL_REPO_ID = "Disha252001/tourism-1-models"   # <--- update if different
MODEL_FILENAME = "models/GradientBoosting_best_model.joblib"  # change to whichever model file you uploaded

# Hugging Face token (should be set in env for private models)
HF_TOKEN = os.environ.get("HF_TOKEN", None)
if HF_TOKEN is None:
    st.warning("HF_TOKEN not found in environment — public models will still download, private ones will fail.")

# Local cache path for downloaded model
cache_dir = Path.home() / ".cache" / "hf_models"
cache_dir.mkdir(parents=True, exist_ok=True)

@st.cache_resource
def load_model():
    try:
        # Download model file from Hugging Face Hub to local cache
        model_path = hf_hub_download(
            repo_id=MODEL_REPO_ID,
            filename=MODEL_FILENAME,
            repo_type="model",
            token=HF_TOKEN,
            cache_dir=str(cache_dir)
        )
        model = joblib.load(model_path)
        return model
    except Exception as e:
        st.error(f"Failed to load model from Hugging Face Hub: {e}")
        raise

model = load_model()

st.title("Wellness Tourism — Purchase Prediction")
st.write("Fill the customer details below and click **Predict**.")

# --------------------------
# Input form: fields based on your data dictionary
# Fill or remove fields to match your training data exactly
# --------------------------
with st.form("input_form"):

    Age = st.number_input("Age", min_value=0, max_value=120, value=35)
    TypeofContact = st.selectbox("Type of Contact", ["Company Invited", "Self Inquiry"])
    CityTier = st.selectbox("City Tier", [1, 2, 3], index=1)
    Occupation = st.selectbox("Occupation", ["Salaried", "Freelancer", "Business", "Other"])
    Gender = st.selectbox("Gender", ["Male", "Female"])
    NumberOfPersonVisiting = st.number_input("Number Of Person Visiting", min_value=0, value=2)
    PreferredPropertyStar = st.number_input("Preferred Property Star", min_value=1, max_value=7, value=5)
    MaritalStatus = st.selectbox("Marital Status", ["Single", "Married", "Divorced"])
    NumberOfTrips = st.number_input("Number Of Trips (annual)", min_value=0, value=2)
    Passport = st.selectbox("Passport (0=No,1=Yes)", [0,1], index=1)
    OwnCar = st.selectbox("Own Car (0=No,1=Yes)", [0,1], index=1)
    NumberOfChildrenVisiting = st.number_input("Number Of Children Visiting (below 5)", min_value=0, value=0)
    Designation = st.text_input("Designation", value="Manager")
    MonthlyIncome = st.number_input("Monthly Income", min_value=0, value=50000)
    PitchSatisfactionScore = st.number_input("Pitch Satisfaction Score (1-10)", min_value=0, max_value=10, value=8)
    ProductPitched = st.selectbox("Product Pitched", ["Wellness Package", "Family Package", "Other"])
    NumberOfFollowups = st.number_input("Number Of Followups", min_value=0, value=1)
    DurationOfPitch = st.number_input("Duration Of Pitch (minutes)", min_value=0, value=10)

    submitted = st.form_submit_button("Predict")

# --------------------------
# Convert inputs to DataFrame and predict
# --------------------------
def build_input_df():
    row = {
        "Age": Age,
        "TypeofContact": TypeofContact,
        "CityTier": CityTier,
        "Occupation": Occupation,
        "Gender": Gender,
        "NumberOfPersonVisiting": NumberOfPersonVisiting,
        "PreferredPropertyStar": PreferredPropertyStar,
        "MaritalStatus": MaritalStatus,
        "NumberOfTrips": NumberOfTrips,
        "Passport": Passport,
        "OwnCar": OwnCar,
        "NumberOfChildrenVisiting": NumberOfChildrenVisiting,
        "Designation": Designation,
        "MonthlyIncome": MonthlyIncome,
        "PitchSatisfactionScore": PitchSatisfactionScore,
        "ProductPitched": ProductPitched,
        "NumberOfFollowups": NumberOfFollowups,
        "DurationOfPitch": DurationOfPitch
    }
    df = pd.DataFrame([row])
    return df

if submitted:
    input_df = build_input_df()
    st.subheader("Input (as DataFrame)")
    st.dataframe(input_df)

    # Model should be a pipeline with preprocessor so we can directly pass df
    try:
        pred = model.predict(input_df)
        proba = None
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(input_df)
            # If binary, proba[:,1] is probability of class 1
            if proba.shape[1] == 2:
                proba = float(proba[:,1][0])
            else:
                # multi-class: show list
                proba = [float(x) for x in proba[0]]
        st.success(f"Prediction (ProdTaken): {int(pred[0])}")
        if proba is not None:
            st.info(f"Probability: {proba}")
    except Exception as e:
        st.error(f"Failed to predict: {e}")


Overwriting master_folder/deployment/app.py


In [37]:
%%writefile master_folder/deployment/push_to_space.py
import os
from huggingface_hub import create_repo, upload_file
from pathlib import Path

HF_TOKEN = os.environ.get("HF_TOKEN")
if not HF_TOKEN:
    raise RuntimeError("HF_TOKEN environment variable is required.")

SPACE_ID = "Disha252001/wellness-tourism-space"  
LOCAL_DEPLOY_DIR = Path(__file__).resolve().parent  

# 1) Create Space as DOCKER TYPE (correct)
try:
    create_repo(
        repo_id=SPACE_ID,
        token=HF_TOKEN,
        repo_type="space",
        private=False,
        space_sdk="docker"      # <-- FIXED
    )
    print(f"Space created: {SPACE_ID}")
except Exception as e:
    print(f"create_repo info (safe to ignore if exists): {e}")

# 2) Upload files
files_to_upload = ["Dockerfile", "requirements.txt", "app.py", ".gitignore"]

for fname in files_to_upload:
    local_path = LOCAL_DEPLOY_DIR / fname
    if not local_path.exists():
        print(f"Skipping {fname} — not found.")
        continue

    print(f"Uploading {fname}...")
    upload_file(
        path_or_fileobj=str(local_path),
        path_in_repo=fname,
        repo_id=SPACE_ID,
        repo_type="space",
        token=HF_TOKEN
    )

print(f"Done! Visit your Space:")
print(f"https://huggingface.co/spaces/{SPACE_ID}")


Overwriting master_folder/deployment/push_to_space.py


In [20]:
%%writefile master_folder/deployment/.gitignore
__pycache__/
*.pyc
*.pkl
*.joblib
.env
*.egg-info


Writing master_folder/deployment/.gitignore
